In [1]:
#imports

import cv2
from cv2 import threshold
import numpy as np
import tracker.tracker as tracker
%load_ext autoreload
%autoreload 2

In [2]:
# initial setup

threshold = 40
webcam = cv2.VideoCapture(1)
ret, frame = webcam.read()

# Will's webcam dims are: 480, 640, 3
height, width, d = frame.shape
print(height, width)

#this should turn drawing area into square
line_x = width - height
box_width = 100 #arbitrarily set

xcenter = 500
ycenter = 400
xcenter_bw = xcenter - (width - height)
ycenter_bw = ycenter
# this is used to creat the test data
# set to false when not recording webcam
recording = False
if recording:
    out = cv2.VideoWriter("Test_video2.avi", cv2.VideoWriter_fourcc(*'MJPG'), 10,(width, height)) 


sync_finger = None
sync_finger_bw = None
sync_frame = None
next_frame = None
while(True):
		
    # Capture the video frame
    # by frame
    if next_frame is None:
        ret, frame = webcam.read()
        _, next_frame = webcam.read()
    else:
        frame = next_frame
        _, next_frame = webcam.read()


    # flip so it makes sense to the user for drawing
    # frame = cv2.flip(frame, 1)
    if recording:
        out.write(frame)

    # duplicate current frame
    # one is used for processing the other is show to user
    c_frame = frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # sync with background by pressing 's'
    if cv2.waitKey(1) & 0xFF == ord('s'):
        sync_frame = frame[:, line_x:]

    # offset from center point to find the region
    offset = int(round(box_width/2))
    if sync_frame is not None:

        #  remove background from current frame
        cut_frame =  frame[:, line_x:]
        cut_frame = cv2.subtract(sync_frame, cut_frame)


        # create binary image
        im_bw = cv2.threshold(cut_frame, threshold, 255, cv2.THRESH_BINARY)[1]


        if cv2.waitKey(1) & 0xFF == ord('f'):
            sync_finger = frame[ycenter-offset:ycenter+offset, xcenter-offset:xcenter+offset]
            sync_finger_bw = im_bw[ycenter_bw-offset:ycenter_bw+offset, xcenter_bw-offset:xcenter_bw+offset]


        cv2.imshow('bin', im_bw)


    # Display the resulting frame
    # this will be what the user ends up seeing
    cv2.line(c_frame, (line_x, -10), (line_x, height + 10), (0, 255, 0), thickness=3)
    if sync_finger is None and sync_frame is not None or recording:
            cv2.rectangle(c_frame, (xcenter-offset, ycenter-offset), (xcenter+offset, ycenter+offset), (0, 200, 200), 5)

    cv2.imshow('frame', c_frame)

    if sync_finger is not None:
        cv2.imshow("finger tip", sync_finger)
        cv2.imshow("finger tip bw", sync_finger_bw)

        """        
            begin processing here
            at this point the fingertip has been submitted
            the corrdinates of the finger are below
            the region taken by the fingertip is rows 215:265  cols 375:425 in the normal image
            the region taken by the fingertip is rows 215:265  cols 215:265 in the bw image
        """

        regions = [(ycenter, xcenter, box_width),
                    ]
        new_locations = tracker.track_regions(frame, next_frame, regions=regions, tracking_scale=0.9)
        print(f"New: {new_locations}")
        coords = new_locations[0]
        ycenter = coords[0]
        xcenter = coords[1]

        cv2.circle(c_frame, (coords[1], coords[0]), 10, (255, 0, 0), 3)

    cv2.imshow('frame', c_frame)



# After the loop release the cap object
webcam.release()
if recording:
    out.release()

# Destroy all the windows
cv2.destroyAllWindows()



1080 1920
True
New: [(400, 499, 100)]
True
New: [(400, 498, 100)]
True
New: [(400, 497, 100)]
True
New: [(400, 496, 100)]
True
New: [(400, 494, 100)]
True
New: [(400, 494, 100)]
True
New: [(400, 493, 100)]
True
New: [(400, 491, 100)]
True
New: [(402, 490, 100)]
True
New: [(403, 489, 100)]
True
New: [(405, 489, 100)]
True
New: [(407, 488, 100)]
True
New: [(408, 487, 100)]
False
New: [(410, 488, 100)]
False
New: [(412, 488, 100)]
True
New: [(412, 487, 100)]
True
New: [(409, 486, 100)]
True
New: [(407, 484, 100)]
True
New: [(406, 482, 100)]
True
New: [(405, 481, 100)]
True
New: [(404, 480, 100)]
True
New: [(403, 479, 100)]
True
New: [(402, 478, 100)]
True
New: [(401, 477, 100)]
True
New: [(401, 476, 100)]
True
New: [(401, 475, 100)]
True
New: [(403, 474, 100)]
False
New: [(404, 474, 100)]
False
New: [(406, 474, 100)]
False
New: [(407, 475, 100)]
True
New: [(410, 474, 100)]
False
New: [(412, 474, 100)]
False
New: [(413, 475, 100)]
False
New: [(415, 476, 100)]
True
New: [(418, 476, 100)]
Tr